## Libraries Required:
This Project uses only Keras for all purposes including a pre-trained model and Optimization.

In [ ]:
import numpy as np
from time import time
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.layers import Input
import keras.backend as K
from matplotlib.pyplot import imshow, show, imsave
%matplotlib inline

## Input Images:

In [ ]:
style_image = "style/image/path"          #input your style image path
content_image = "content/image/path"      #input your content image path

## Helper Functions:
Functions for Preprocesssing input, Losses and Regenerating the Stylized Image from Outputs.

In [ ]:
def preprocess(image_path):
    """Function for loading and preprocessing input images.
    This function uses VGG16's inbuilt preprocessing function.
    
    #Arguments
        image_path: Path for the image.
        
    #Returns
        Processed Image array with zero mean.
    """
    img = image.load_img(image_path, target_size=(600,800))
    img = image.img_to_array(img)
    img = preprocess_input(img)           #preprocess_input function from vgg
    img = np.expand_dims(img, axis=0)     #CNN input requires 4-D arrays of 
                                          #shape (batch_size, height, width, channels)
    return img

In [ ]:
def deprocess_image(x):
    """Function for regenerating stylized image from Model Output
    Since the Images were zero-centered and were converted to 
    float, we need a deprocessing function to retrieve a displayable
    image. 
    
    #Arguments
        x: A 4-D array containing a single processed image.
        
    #Returns
        x: A 3-D array containing the output image.
    """
    x = x[0]
    x[:, :, 0] += 103.939                 #These are the approximate means,
    x[:, :, 1] += 116.779                 #precomuted channel-wise.
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')#integer type for image array
    return x

In [ ]:
def gram_matrix(A):
    """Function for computing correlation between different output
       channels.
    
    #Arguments
        A: A 4-D tensor output of a layer in the model.
        
    #Returns
        Gram-matrix of A.
    """
    return K.dot(A, K.transpose(A))

In [ ]:
def content_cost():
    """Function for computing the dissimilarity between Generated
       image and input content image.
    This uses normalised Mean Square Error (MSE).
    
    #Arguments
        None
        
    #Returns
        Computed Content Cost in a tensor.
    """
    J_content = K.variable(0.)
    
    a_G = model.layers[12].output
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    a_G_unrolled = K.reshape(a_G,(-1, n_H*n_W, n_C))
    
    J_content = (1/(4*n_H*n_W*n_C))*K.sum(K.square(a_C_unrolled-a_G_unrolled))
    return J_content

In [ ]:
def compute_layer_style_cost(a_S, a_G):
    """Function for computing dissimilarity between Style Image
       output and Generated Image at a given layer.
    
    #Arguments
        a_S: Style Image Activations
        a_G: Generated Image Activation
    
    #Returns
        J_style_layer: Computed cost at a layer in a tensor
    """
    
    J_style_layer = K.variable(0.)
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
   
    a_S = K.transpose(K.reshape(a_S, (n_H*n_W,n_C)))
    a_G = K.transpose(K.reshape(a_G, (n_H*n_W,n_C)))

    GS = gram_matrix(a_S)
    GG = gram_matrix(a_G)


    J_style_layer = K.square(1/(2*n_C*n_H*n_W))*K.sum(K.sum(K.square(GS-GG)))
    
    return J_style_layer

In [ ]:
def compute_style_cost(model=model, STYLE_LAYERS=style_layers):
    """Function for computing Average Style Cost across the layers.
    
    #Arguments
        model: Keras Model Instance
        STYLE_LAYERS: List of Style Layer indices to be included in 
                      calculation.
    
    #Returns
        J_style: Computed Style Cost across layers in a variable tensor.
    """
    
    J_style = K.variable(0.)
    i=0

    for layer in STYLE_LAYERS:
        
        a_G = model.layers[layer].output

        J_style_layer = compute_layer_style_cost(style_features[i], a_G)

        J_style += 0.25 * J_style_layer
        i+=1

    return J_style

In [ ]:
def total_cost(alpha = 1000, beta = 4):
    """Function for computing Final Cost as a weighted average of 
       Style and Content Costs.
    
    #Arguments
        alpha: Weight of content cost
        beta: weight of style cost
    
    #Returns
        J: weighted sum of style and content cost
    """

    J = K.variable(0.)
    J = (alpha*content_cost()+beta*compute_style_cost())
    
    return J

## Setting Up the Graph:

In [ ]:
style_image = preprocess(style_image)
content_image = preprocess(content_image)
gen_image = K.variable(np.random.randn(1,600,800,3))

In [ ]:
print("style_image ",style_image.shape)
print("content_image ",content_image.shape)
print("gen_image ",gen_image.shape)

In [ ]:
style_layers = [1,4,7,11]               #Inclusion of more layers produces
                                        #better results at the cost of 
                                        #computational complexity
                                        #try [1,4,7,11,15]
content_layers = [12]

### Pre-Trained Keras VGG16 Model

In [ ]:
model = VGG16(include_top=False, weights='imagenet', input_tensor=Input(tensor=gen_image))
model.summary()

### Extracting Style and Content Activations for Input Images

In [ ]:
style_features = []
for layer in style_layers:
    fn = K.function([model.layers[0].input], [model.layers[layer].output])
    style_features.append(fn([style_image])[0])

In [ ]:
content_features = []
for layer in content_layers:
    fn = K.function([model.layers[0].input], [model.layers[layer].output])
    content_features.append(fn([content_image])[0])    

In [ ]:
for _ in style_features:
    print(_.shape)

In [ ]:
content_features[0].shape

In [ ]:
m, w, h, c = a_C.shape()
a_C_unrolled = K.reshape(content_features[0], (-1,w*h,c))
del content_features
print("a_C_unrolled ", a_C_unrolled.shape)

### Loss Metric

In [ ]:
total_loss = total_cost()

### Training Step

In [ ]:
opt = Adam(lr=2.0)
updates = opt.get_updates([gen_image],[],total_loss)
train = K.function([],[total_loss],updates)
print("Training Step defined.")

## Optimisation

In [ ]:
start = time()
n_epochs = 1000
for epoch in range(n_epochs):
    e_start = time()
    out = train([])
    e_end = time()
    print("Epoch: {}, Loss: {:.2e}, Time taken per Step: {:.2f}".format(epoch, out[0], e_end-e_start))
    if(epoch%20==0):
        imshow(deprocess_image(K.get_value(gen_image)))
        show()
        print("ETA: {}".format((e_end-start)*(500-epoch)/(epoch+1)))
print("Total time taken: {:.2f}".format(time()-start))

## Saving the Result

In [ ]:
final_out = deprocess_image(K.get_value(gen_image))
imsave(final_out,"styled_image.jpg")